In [1]:
import random
import time
import threading
import pygame

from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker

ModuleNotFoundError: No module named 'pygame'

In [2]:
image_path = 'canvas.jpg'


class Canvas:
    screen_width = 1050
    screen_height = 680

    def __init__(self):
        self.screen = pygame.display.set_mode((Canvas.screen_width, Canvas.screen_height))
        self.bg_img = pygame.image.load(image_path)
        self.ball_color = pygame.Color(255, 0, 0)
        self.old_ball_rect = None

        self.screen.blit(self.bg_img, (0, 0))
        pygame.display.flip()

    @classmethod
    def recalc_coords(cls, coords):
        screen_x = round(cls.screen_width / 100 * float(coords[0]), 0)
        screen_y = round(cls.screen_height / 100 * float(coords[1]), 0)
        return (screen_x,screen_y)

    def move_ball(self, coords):
        screen_coords = Canvas.recalc_coords(coords)
        if self.old_ball_rect:
            self.screen.blit(self.bg_img,self.old_ball_rect.topleft,area=self.old_ball_rect)
            pygame.display.update(self.old_ball_rect)
        new_ball_rect = pygame.draw.circle(self.screen,self.ball_color,screen_coords, 4)
        pygame.display.update(new_ball_rect)
        self.old_ball_rect = new_ball_rect

In [3]:
engine = create_engine('postgresql+psycopg2://develop:}c%4Z>n~M<3p:Em\@localhost/whoscored')
Session = sessionmaker(bind=engine)
Base = automap_base()
Base.prepare(engine, reflect=True,schema='core')
session = Session()
Event = Base.classes.events
EType = Base.classes.event_types

In [4]:
event_types = {etype.typeid:etype.typetext for etype in session.query(EType.typeid, EType.typetext)}
game_ids = tuple({result[0] for result in session.query(Event.globalgameid).limit(1)})
game_id = random.choice(game_ids)
events = tuple({result for result in session.query(Event).where(Event.globalgameid==game_id)})

In [5]:
events = sorted(events, key=lambda event:(event.time, event.globaleventid))

In [15]:
def data_feed(events, time_scale = 1, screen = False):
    if not events:
        return
    home_team_id = events[0].teamid
    elapsed_time = 0
    for event in events:
        if not running:
            print('window down, data feed stoped')
            return    
        
        time_diff = event.time - elapsed_time
        if time_diff > 0:
            time.sleep(time_diff / time_scale)
            elapsed_time += time_diff
        print(f'{event.time // 60}:{event.time%60} - {event.teamid} - {event_types[event.type]}')
        coords = (event.x, event.y) if event.teamid == home_team_id else (100 - event.x, 100 - event.y)
        if screen:
            canvas.move_ball(coords)
        
        if event.endx:
            distance = (float((event.endx - event.x))**2 + float((event.endy - event.y))**2)**0.5
            if distance > 20:
                ball_speed = 20
            elif distance > 40:
                ball_speed = 23
            else:
                ball_speed = 17
            ball_moving_time = distance / ball_speed
            time.sleep(ball_moving_time / time_scale)
            elapsed_time += ball_moving_time
            #print(f'Ball movement took {round(ball_moving_time,1)}')
            coords = (event.endx, event.endy) if event.teamid == home_team_id else (100 - event.endx, 100 - event.endy)
            if screen:
                canvas.move_ball(coords)

In [10]:
canvas = Canvas()
running = True

data_thread = threading.Thread(target=data_feed,args=(events,), kwargs = {'time_scale' : 1})
data_thread.start()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

0:0 - 148 - FormationSet
0:0 - 148 - Start
0:0 - 145 - FormationSet
0:0 - 145 - Start
0:1 - 148 - Pass
Ball movement elapsed 0.861719533046117
0:3 - 148 - Pass
Ball movement elapsed 2.2574432440263035
0:7 - 148 - Pass
Ball movement elapsed 0.46407253937214166
window down, data feed stoped


In [16]:
running = True
data_feed(events, time_scale = 100000000000000)

0:0 - 148 - FormationSet
0:0 - 148 - Start
0:0 - 145 - FormationSet
0:0 - 145 - Start
0:1 - 148 - Pass
0:3 - 148 - Pass
0:7 - 148 - Pass
0:8 - 148 - Pass
0:10 - 148 - Pass
0:12 - 148 - Pass
0:14 - 148 - Pass
0:17 - 148 - Pass
0:20 - 148 - Pass
0:21 - 148 - Pass
0:25 - 148 - Pass
0:32 - 148 - Pass
0:34 - 148 - Pass
0:36 - 145 - BallRecovery
0:45 - 145 - Pass
0:48 - 145 - Pass
0:50 - 145 - Pass
0:54 - 145 - Pass
0:56 - 145 - Pass
0:58 - 145 - Pass
1:1 - 145 - Pass
1:5 - 145 - Pass
1:13 - 145 - Pass
1:17 - 145 - Pass
1:19 - 145 - Pass
1:22 - 145 - Pass
1:23 - 145 - Pass
1:53 - 148 - Pass
1:58 - 145 - Pass
2:0 - 145 - BallRecovery
2:1 - 148 - Challenge
2:1 - 145 - TakeOn
2:4 - 145 - Foul
2:4 - 148 - Foul
2:17 - 145 - Pass
2:18 - 145 - Pass
2:20 - 145 - Pass
2:24 - 145 - Pass
2:29 - 148 - BallRecovery
2:30 - 145 - Tackle
2:30 - 148 - TakeOn
2:31 - 145 - BallTouch
2:32 - 145 - BallTouch
2:35 - 148 - Pass
2:37 - 145 - BallRecovery
2:39 - 145 - Pass
2:40 - 145 - BallTouch
3:13 - 148 - Pass
3:1

25:52 - 145 - Pass
25:53 - 148 - Clearance
25:55 - 148 - Clearance
25:57 - 145 - Pass
25:59 - 145 - Aerial
25:59 - 148 - Aerial
26:0 - 145 - Goal
27:6 - 148 - Pass
27:8 - 148 - Pass
27:11 - 148 - Pass
27:13 - 148 - Pass
27:21 - 148 - BallTouch
27:23 - 148 - Foul
27:23 - 145 - Foul
27:35 - 145 - Pass
27:37 - 145 - Pass
27:38 - 145 - Pass
27:39 - 145 - Pass
27:42 - 145 - Pass
27:47 - 145 - Pass
27:53 - 145 - Pass
27:53 - 148 - Clearance
27:54 - 145 - CornerAwarded
27:54 - 148 - CornerAwarded
28:19 - 145 - Pass
28:21 - 148 - Punch
28:24 - 145 - Pass
28:26 - 148 - BlockedPass
28:26 - 145 - Pass
28:38 - 145 - Pass
28:40 - 145 - BallTouch
28:52 - 148 - Pass
28:54 - 145 - Pass
28:55 - 145 - BallRecovery
28:56 - 145 - Pass
28:59 - 145 - Pass
29:3 - 145 - Pass
29:6 - 145 - Pass
29:9 - 145 - Pass
29:12 - 145 - Pass
29:20 - 145 - Pass
29:21 - 145 - Pass
29:24 - 145 - Pass
29:25 - 145 - Pass
29:27 - 145 - Pass
29:28 - 145 - Pass
29:29 - 145 - Pass
29:31 - 145 - Pass
29:33 - 145 - Pass
29:36 - 148 

49:34 - 145 - Pass
49:35 - 145 - Pass
49:36 - 148 - Interception
49:37 - 148 - BallRecovery
49:38 - 148 - Pass
49:40 - 148 - Pass
49:44 - 148 - Pass
49:47 - 148 - Pass
49:49 - 148 - Pass
49:51 - 148 - Pass
49:53 - 148 - MissedShots
50:28 - 145 - Pass
50:31 - 145 - Aerial
50:31 - 148 - Aerial
50:32 - 148 - Pass
50:33 - 145 - Aerial
50:33 - 148 - Aerial
50:34 - 145 - Pass
50:35 - 148 - BallRecovery
50:36 - 148 - Pass
50:37 - 148 - Pass
50:37 - 145 - BlockedPass
50:38 - 145 - BallRecovery
50:39 - 145 - Pass
50:42 - 145 - Pass
50:44 - 148 - BallTouch
50:45 - 145 - Pass
50:49 - 148 - BlockedPass
50:49 - 145 - Pass
50:52 - 148 - BallRecovery
50:53 - 148 - Pass
50:55 - 145 - Tackle
50:55 - 148 - Dispossessed
50:58 - 145 - BallRecovery
51:0 - 145 - Pass
51:3 - 145 - Pass
51:4 - 145 - Pass
51:13 - 145 - Pass
51:16 - 145 - Pass
51:44 - 148 - Pass
51:45 - 145 - Pass
51:46 - 145 - Aerial
51:46 - 148 - Aerial
51:47 - 148 - Pass
51:50 - 148 - Pass
51:50 - 145 - BlockedPass
51:51 - 145 - BallRecovery

78:29 - 145 - BallTouch
78:39 - 148 - Pass
78:42 - 148 - Pass
78:48 - 148 - Pass
78:53 - 148 - Pass
78:55 - 148 - Pass
78:59 - 148 - Pass
79:0 - 148 - Pass
79:5 - 148 - Pass
79:7 - 148 - Pass
79:10 - 148 - Pass
79:13 - 148 - Pass
79:17 - 145 - Pass
79:18 - 145 - BallRecovery
79:19 - 145 - Pass
79:23 - 148 - Foul
79:23 - 145 - Foul
79:28 - 148 - Card
79:55 - 145 - Pass
79:56 - 145 - Pass
80:0 - 148 - Foul
80:0 - 145 - Foul
80:39 - 145 - Pass
80:41 - 145 - Aerial
80:41 - 148 - Aerial
80:42 - 148 - Clearance
80:43 - 145 - Pass
80:44 - 148 - Clearance
80:46 - 148 - BallTouch
80:49 - 145 - Pass
80:52 - 145 - Pass
80:53 - 148 - Clearance
80:59 - 148 - Tackle
80:59 - 145 - TakeOn
81:14 - 145 - Pass
81:15 - 145 - Pass
81:16 - 145 - Pass
81:18 - 145 - Pass
81:19 - 145 - Dispossessed
81:19 - 148 - Tackle
81:21 - 148 - BallRecovery
81:22 - 148 - Pass
81:23 - 148 - Pass
81:26 - 148 - Pass
81:28 - 145 - BallRecovery
81:29 - 145 - Pass
81:30 - 145 - Pass
81:31 - 145 - Pass
81:36 - 145 - Foul
81:36 -